In [1]:
import pandas as pd
import numpy as np
import os 
import omegaconf
import hydra
from diagnostics.io import get_base_config

In [2]:
from diagnostics.inventory import ModelInventoryBuilder, QueryBuilder

Steps:
1. load our hydra cfg (start with 1 and extend)
2. flatten hierarchy.
3. make into a df (optionally append more).
4. build queries.


In [3]:
model_inventory = ModelInventoryBuilder("/home/jovyan/grid_artifacts")

In [4]:
total_df = model_inventory.build_dframe()

Building model registry from 789 configs...


100%|██████████| 789/789 [00:28<00:00, 27.98it/s]


In [6]:
query_builder = QueryBuilder(total_df)
# query_builder.add_query("dali.context.train.consecutive_sequences", "==", "True")
query_builder.add_query("training.rng_seed_data_pt", "==", "0")
query_builder.add_query("model.losses_to_use", "==", "[pca_singleview]")
query_builder.add_timestamp_query("2022-07-15", "2022-07-20")
print(query_builder.combine_queries("and"))

(`training.rng_seed_data_pt` == '0') and (`model.losses_to_use` == '[pca_singleview]') and (timestamp >= '2022-07-15' and timestamp <= '2022-07-20')


In [7]:
queried_df = total_df.query(query_builder.combine_queries("and"))

In [8]:
queried_df.shape

(80, 97)

In [186]:
queried_df["timestamp"]
#queried_df.query("timestamp > `2020-07-19`")

0   2022-07-20 21:40:42
0   2022-07-20 21:41:35
0   2022-07-20 21:38:58
0   2022-07-20 21:44:53
0   2022-07-20 21:39:48
0   2022-07-20 21:40:49
0   2022-07-20 21:44:45
0   2022-07-20 21:43:02
0   2022-07-20 21:41:27
0   2022-07-20 21:44:03
0   2022-07-20 21:39:57
0   2022-07-20 21:39:06
0   2022-07-20 21:43:54
0   2022-07-20 21:42:24
0   2022-07-20 21:43:10
0   2022-07-20 21:42:17
Name: timestamp, dtype: datetime64[ns]